In [1]:
import tensorflow as tf

loss_fn = tf.keras.losses.mean_squared_error
print(loss_fn(tf.ones((1, 2,)), tf.Variable(tf.zeros((1, 2)))))

y_pred = tf.Variable((2, 2), dtype = "float32")
with tf.GradientTape() as tape:
    #tape.watch(y_pred) # benötigt, wenn y_pred = tf.constant(...)
    loss_value = loss_fn(tf.ones((1, 2,)), y_pred)
    
gradients = tape.gradient(loss_value, y_pred)
print(gradients)
# das passt. [1, 1] ist die Ableitung des MSE ausgewertet in y_pred_1 = 2 und y_pred_2 = 2

tf.Tensor([1.], shape=(1,), dtype=float32)
tf.Tensor([1. 1.], shape=(2,), dtype=float32)


In [2]:
loss_value

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>

In [10]:
import tensorflow as tf
import numpy as np

loss_fn = tf.keras.losses.mean_squared_error
print(loss_fn(tf.ones((2, 2,)), tf.Variable(tf.zeros((2, 2)))))

y = np.array([1, 1])
y_pred = np.array([2, 2])

y = tf.constant(y, dtype = "float32")
y_pred = tf.Variable(y_pred, dtype = "float32")
#y_pred = tf.Variable((2,2), dtype = "float32")
with tf.GradientTape() as tape:
    # tape.watch(y_pred) # benötigt, wenn y_pred = tf.constant(...)
    loss_value = loss_fn(y, y_pred)
    
gradients = tape.gradient(loss_value, y_pred)
print(gradients)
# das passt. [1, 1] ist die Ableitung des MSE ausgewertet in y_pred_1 = 2 und y_pred_2 = 2

tf.Tensor([1. 1.], shape=(2,), dtype=float32)
tf.Tensor([1. 1.], shape=(2,), dtype=float32)


In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution() # macht keinen Unterschied
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

tf.random.set_seed(0)

model_in = keras.Input(shape=(2,))
model_d1 = layers.Dense(64, activation='relu')(model_in)
model_d2 = layers.Dense(64, activation='relu')(model_d1)
model_d3 = layers.Dense(1)(model_d2)
model = keras.Model(inputs=[model_in], outputs=[model_d3])
model.summary()

v = tf.Variable([[0.1, 0.2]], name='v_var')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 64)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,417
Trainable params: 4,417
Non-trainable params: 0
_________________________________________________________________


In [2]:
with tf.GradientTape() as tape:
    tape.watch(model.input)
    model_vals = model(v)
print(len(tape.watched_variables())) 
# 7      ( = (1 kernel + 1 bias) * (3 dense layers) + (1 v_var) )
print(model.input)
# Tensor("input_1:0", shape=(None, 2), dtype=float32)
model_grad = tape.gradient(model_vals, model.input)
print(model_grad)
# None

7
Tensor("input_1:0", shape=(None, 2), dtype=float32)
None


In [3]:
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(model.input)
    model_vals = model(v)
print(len(tape.watched_variables())) 
# 0      (model.input isn't a variable so it is ignored)

0


In [4]:
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(v)
    model_vals = model(v)
print(len(tape.watched_variables())) 
# 1
print(v)
# <tf.Variable 'v_var:0' shape=(1, 2) dtype=float32, numpy=array([[0.1, 0.2]], dtype=float32)>
model_grad = tape.gradient(model_vals, v)
print(model_grad)
# tf.Tensor([[-0.04392226 -0.06807809]], shape=(1, 2), dtype=float32)

1
<tf.Variable 'v_var:0' shape=(1, 2) dtype=float32>
Tensor("gradient_tape/model_2/dense/MatMul:0", shape=(1, 2), dtype=float32)
